In [ ]:
# preprocess data 

In [9]:
# performing brain extraction using optiBET (brain extraction tool)
# make sure that optiBET shell script is in the directory as the files 
import subprocess 

input_images = ['testing/2015_july.nii.gz', 
                'testing/2017_sept.nii.gz',
                'testing/2021_june.nii.gz',
                'testing/2021_aug.nii.gz']

for input_image in input_images:
    optibet_command = ['sh', 'optiBET.sh', '-i', input_image]
    subprocess.run(optibet_command, check=False)

print('done')


testing/2017_sept is input image
for subject testing/2017_sept.nii.gz use bet for step 1 and MNI152_T1_1mm_brain_mask.nii.gz for MNI mask
step1 BET -B -f 0.1 subject testing/2017_sept for initial extraction
step2 flirt subject testing/2017_sept to MNI space
step3 fnirt subject testing/2017_sept to MNI space
step4 quality control of fnirt using applywarp to put subject testing/2017_sept in MNI space
step5 invert nonlinear warp for subject testing/2017_sept
step6 apply inverted nonlinear warp to MNI label: MNI152_T1_1mm_brain_mask for subject testing/2017_sept
step 7 creating binary brain mask for subject testing/2017_sept
step 8 creating brain extraction for subject testing/2017_sept
removing intermediate files
testing/2021_june is input image
for subject testing/2021_june.nii.gz use bet for step 1 and MNI152_T1_1mm_brain_mask.nii.gz for MNI mask
step1 BET -B -f 0.1 subject testing/2021_june for initial extraction
step2 flirt subject testing/2021_june to MNI space
step3 fnirt subject te

In [14]:
# run SIENA analyses 
# comment out the ones you don't want before running 

import subprocess
import time

# files w/unextracted brains 
normal_files = [('2015_july.nii.gz', '2017_sept.nii.gz'), # 7/2015 ~ 9/2017
              ('2017_sept.nii.gz', '2021_june.nii.gz'), # 9/2017 ~ 6/2021
              ('2021_june.nii.gz', '2021_aug.nii.gz'), # 6/2021 ~ 8/2021
              ('2015_july.nii.gz', '2021_aug.nii.gz')] # overall change 

# files w/brains extracted using optiBET
optibet_files = [('2015_july_optiBET_brain.nii.gz', '2017_sept_optiBET_brain.nii.gz'), # 7/2015 ~ 9/2017
              ('2017_sept_optiBET_brain.nii.gz', '2021_june_optiBET_brain.nii.gz'), # 9/2017 ~ 6/2021
              ('2021_june_optiBET_brain.nii.gz', '2021_aug_optiBET_brain.nii.gz'), # 6/2021 ~ 8/2021
              ('2015_july_optiBET_brain.nii.gz', '2021_aug_optiBET_brain.nii.gz')] # overall change 

def run_analysis(file_directory, files, optiBET = False, ventricles = False):
    runtimes = []

    for file in files:
        input1, input2 = file
        input1_full, input2_full = f'{file_directory}/{input1}', f'{file_directory}/{input2}'

        if optiBET: # disabling BET and setting output directory to specify optiBET was used
            output_directory = f'{input1}_{input2}_optiBET'
            siena_command = ['siena', input1_full, input2_full, '-B', "", '-o', output_directory]
        elif ventricles: # enabling ventricle option and setting output directory to specify 
            output_directory = f'{input1}_{input2}_ventricle'
            siena_command = ['siena', input1_full, input2_full, '-V']
        else: # normal SIENA analysis
            output_directory = f'{input1}_{input2}_normal'
            siena_command = ['siena', input1_full, input2_full]
            
        start_time = time.time()
        completed_process = subprocess.run(siena_command, check=False)
        end_time = time.time() 
        duration = end_time - start_time
        runtimes.append(duration)
        
        return_code = completed_process.returncode
        if return_code == 0:
            print(f'analysis for {input1} and {input2} completed in {duration} seconds!')
        else:
            print(f'analysis for {input1} and {input2} failed with return code {return_code}')


    print('finished with all analyses!')
    for i in range(len(runtimes)):
        runtime = runtimes[i]
        print(f'Process {i + 1} took {round(runtime // 60)} minutes and {round(runtime % 60)} seconds to complete.')

# # run standard SIENA analysis
# run_analysis('testing', normal_files)

# run SIENA analysis using images extracted with optiBET
run_analysis('testing', optibet_files, True)

# # run VIENA analysis (SIENA but with ventricles)
# run_analysis('testing', normal_files, False, True)




Finished. The SIENA report can be viewed by pointing your web browser at:
file:/Users/rikio/Desktop/ms47/2015_july_optiBET_brain.nii.gz_2017_sept_optiBET_brain.nii.gz_optiBET/report.html
Estimated percentage brain volume change (PBVC) = 
.8781605000

analysis for 2015_july_optiBET_brain.nii.gz and 2017_sept_optiBET_brain.nii.gz completed in 815.3139371871948 seconds!

Finished. The SIENA report can be viewed by pointing your web browser at:
file:/Users/rikio/Desktop/ms47/2017_sept_optiBET_brain.nii.gz_2021_june_optiBET_brain.nii.gz_optiBET/report.html
Estimated percentage brain volume change (PBVC) = 
-.4272340000

analysis for 2017_sept_optiBET_brain.nii.gz and 2021_june_optiBET_brain.nii.gz completed in 416.4461979866028 seconds!

Finished. The SIENA report can be viewed by pointing your web browser at:
file:/Users/rikio/Desktop/ms47/2021_june_optiBET_brain.nii.gz_2021_aug_optiBET_brain.nii.gz_optiBET/report.html
Estimated percentage brain volume change (PBVC) = 
-.0621349020

analy